In [13]:
import pandas as pd

from dotenv import load_dotenv
import os

load_dotenv()

DB_URI = os.getenv('DB_URI_DEV')

In [16]:
pd.read_sql('articles', DB_URI)

,id,url,title,date,body,author
0,1,https://www.inform.kz/ru/eksperty-nazvali-10-r...,Эксперты назвали 10 рисков для глобальной экон...,2012-01-01,АСТАНА. 1 января. КАЗИНФОРМ - Аналитики немецк...,None
1,2,https://www.inform.kz/ru/zapadnoe-polusharie-z...,Западное полушарие Земли встречает Новый год,2012-01-01,АСТАНА. 1 января. КАЗИНФОРМ - Сотни тысяч люде...,None
2,3,https://www.inform.kz/ru/samye-krasivye-i-udac...,Самые красивые и удачливые звезды казахстанско...,2012-01-01,АСТАНА. 1 января. КАЗИНФОРМ - В 2011 году куль...,None
3,4,https://www.inform.kz/ru/v-kazahstane-sostavil...,В Казахстане составили рейтинг самых опасных д...,2012-01-01,"АСТАНА. 1 января. КАЗИНФОРМ - Трассы, проходящ...",None
4,5,https://www.inform.kz/ru/po-dannym-na-31-dekab...,"По данным на 31 декабря 2011г., в Жанаозене 74...",2012-01-01,АСТАНА. 1 января. КАЗИНФОРМ - По данным на 31 ...,None
...,...,...,...,...,...,...
12725,12726,https://www.inform.kz/ru/uvazhaemye-posetiteli...,"Уважаемые посетители сайта, доброе утро!",2012-05-14,Национальное информационное агентство «Казинфо...,None
12726,12727,https://www.inform.kz/ru/glava-gosudarstva-s-o...,Глава государства с официальным визитом прибыл...,2012-05-14,МИНСК. 14 мая. КАЗИНФОРМ - В Минск с официальн...,None
12727,12728,https://www.inform.kz/ru/14-maya-kalendar-kazi...,14 мая. КАЛЕНДАРЬ КАЗИНФОРМА «ДАТЫ. СОБЫТИЯ. И...,2012-05-14,АСТАНА. 14 мая. КАЗИНФОРМ - Казинформ предлага...,None
12728,12729,https://www.inform.kz/ru/investory-vyvodyat-sr...,Инвесторы выводят средства из-за митингов и нефти,2012-05-14,АСТАНА. 14 мая. КАЗИНФОРМ - Россия снова не в ...,None
